In [30]:
# Constants
SEED = 33
TEMP = 0
TIMEOUT = 500

In [7]:
%pip install autogen
%pip install langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import autogen

config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='.env', # If None the function will try to find in the working directory
    filter_dict={
        "model": {
            "gpt-4",
            "gpt-3.5-turbo",
        }
    }
)

In [38]:
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
import os

# Get the current working directory
current_dir = os.getcwd()

# Define discovery output folder
working_directory = os.path.join(current_dir, "output/discovery/")

# Create Langchain toolkit
tools = FileManagementToolkit(
    root_dir=str(working_directory),
).get_tools()

copy_tool, delete_tool, search_tool, move_tool, read_tool, write_tool, list_tool = tools

In [39]:
# Define a function to generate llm_config from a LangChain tool
def generate_llm_config(tool):
    # Define the function schema based on the tool's args_schema
    function_schema = {
        "name": tool.name.lower().replace (' ', '_'),
        "description": tool.description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    if tool.args is not None:
      function_schema["parameters"]["properties"] = tool.args

    return function_schema

# Construct the llm_config
llm_config = {
  "seed": SEED,  # change the seed for different trials
  "temperature": TEMP,
  "timeout": TIMEOUT,
  "config_list": config_list,
  "functions":[
      generate_llm_config(copy_tool),
      generate_llm_config(delete_tool),
      generate_llm_config(search_tool),
      generate_llm_config(move_tool),
      generate_llm_config(read_tool),
      generate_llm_config(write_tool),
      generate_llm_config(list_tool),
  ],
}

In [ ]:
# Agents

# Admin
# A human admin. Interact with the Project Lead to discuss plan. Plan execution needs to be approved by this admin. 

# Project Lead
# Project Lead. Collect project requirements from Admin. Create a plan to accomplish the goals and meets all functional requirements.

# Trading Expert
# Trading Expert. 